13장. 모델 설능 검증하기
- 실험 정확도를 평가하는 방법과 성능을 향상시키는 중요한 머신러닝 기법

In [1]:
import pandas as pd
import numpy as no
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/mulcam_bigdata/data_모두의딥러닝/sonar3.csv'
df = pd.read_csv(path, header = None)
df.head()

# 61개의 열

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,0
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,0
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,0
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,0
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,0


In [4]:
# 암석(0), 광석(1)
print(df.iloc[:,-1])
print()
print(df.iloc[:,-1].value_counts())
# 광석 111개, 암석 97개 >> 총 208개의 샘플이 준비되어 있음

0      0
1      0
2      0
3      0
4      0
      ..
203    1
204    1
205    1
206    1
207    1
Name: 60, Length: 208, dtype: int64

60
1    111
0     97
Name: count, dtype: int64


In [5]:
# 독립변수 (x), 종속변수(y)
x = df.iloc[:, :-1]   # 1~60번째 열을 x 변수에 저장
y = df.iloc[:, -1]    # 광물의 종류는 y로 저장

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 모델 설정
model = Sequential()
model.add(Dense(24, input_dim = 60, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))   # 이진분류는 sigmoid   / 다항분류는 softmax

In [7]:
# 모델 컴파일
model.compile(loss = 'binary_crossentropy', optimizer = 'adam',
              metrics = ['accuracy']) # 이진분류는 binary / 다항분류는 categorical

# 모델 실행
history = model.fit(x, y, epochs = 200, batch_size = 10)

# 200번 반복되었을 때의 정확도 100%

Epoch 1/200
21/21 [==============================] - 1s 3ms/step - loss: 0.6924 - accuracy: 0.5192
Epoch 2/200
21/21 [==============================] - 0s 3ms/step - loss: 0.6652 - accuracy: 0.6587
Epoch 3/200
21/21 [==============================] - 0s 3ms/step - loss: 0.6461 - accuracy: 0.6683
Epoch 4/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6362 - accuracy: 0.7163
Epoch 5/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6216 - accuracy: 0.7212
Epoch 6/200
21/21 [==============================] - 0s 3ms/step - loss: 0.6089 - accuracy: 0.7404
Epoch 7/200
21/21 [==============================] - 0s 3ms/step - loss: 0.5969 - accuracy: 0.7308
Epoch 8/200
21/21 [==============================] - 0s 3ms/step - loss: 0.5838 - accuracy: 0.7452
Epoch 9/200
21/21 [==============================] - 0s 3ms/step - loss: 0.5825 - accuracy: 0.7067
Epoch 10/200
21/21 [==============================] - 0s 4ms/step - loss: 0.5610 - accuracy: 0.7500
Epoch 11/

과적합 이해
- 과적합이란? 모델이 학습 데이터셋 안에서 일정 수준 이상의 예측 정확도를 보임
1. 새로운 데이터에 적용하면 잘 맞지 않는 것
2. 주어진 샘플에만 최적화
3. 그어진 선을 새로운 데이터에 적용하면 정확한 분류가 어려움


---
- 과적합은 층이 너무 많거나 변수가 복잡해서 발생
- 테스트셋과 학습셋이 중복될 때
- 학습단계에서 입력층, 은닉층, 출력층의 노드에 상당히 많은 변수가 투입됨 >> 과적합에 주의



In [8]:
# 과적합 됨을 확인
# 해결방안 : 학습 셋, 테스트 셋 구분

'''
학습셋만 가지고 평가할 때, 층을 더하거나 에포크 값을 높여 실행횟수를 늘리면 정확도가 계속 증가
>> 이러한 예측 성공률이 테스트셋에서도 그대로 나타나지는 않음
>> 학습이 깊어져 학습셋 내부에서 성공률이 높아져도 데스트셋에서는 효과가 없다면 과정합이 일어나고 있음

학습을 진행해도 테스트 결과가 더 이상 좋아지지 않는 지점에서 학습을 멈춤

식이 복잡해지고, 학습량이 늘어날수록 학습 데이터를 통한 예측률은 계속 증가
하지만, 적절하게 조절하지 않을 경우 테스트셋을 이용한 예측률은 오히려 감
'''

from tensorflow.keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = \
train_test_split(x,y, test_size = 0.3, shuffle = True)

In [9]:
# 모델 결정
model = Sequential()
model.add(Dense(24, input_dim = 60, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 24)                1464      
                                                                 
 dense_4 (Dense)             (None, 10)                250       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1725 (6.74 KB)
Trainable params: 1725 (6.74 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
# 모델 컴파일
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# 모델 실행(학습데이터로만) >> 모델 학습
history = model.fit(x_train, y_train, epochs = 200, batch_size = 10)

# >> 학습 데이터셋으로 학습을 진행한 후, 학습의 결과를 저장 >> 모델

Epoch 1/200
15/15 [==============================] - 1s 3ms/step - loss: 0.6828 - accuracy: 0.6345
Epoch 2/200
15/15 [==============================] - 0s 3ms/step - loss: 0.6738 - accuracy: 0.6414
Epoch 3/200
15/15 [==============================] - 0s 4ms/step - loss: 0.6664 - accuracy: 0.6483
Epoch 4/200
15/15 [==============================] - 0s 3ms/step - loss: 0.6572 - accuracy: 0.6828
Epoch 5/200
15/15 [==============================] - 0s 3ms/step - loss: 0.6482 - accuracy: 0.6759
Epoch 6/200
15/15 [==============================] - 0s 3ms/step - loss: 0.6384 - accuracy: 0.6759
Epoch 7/200
15/15 [==============================] - 0s 3ms/step - loss: 0.6285 - accuracy: 0.7172
Epoch 8/200
15/15 [==============================] - 0s 3ms/step - loss: 0.6196 - accuracy: 0.6897
Epoch 9/200
15/15 [==============================] - 0s 3ms/step - loss: 0.6077 - accuracy: 0.7379
Epoch 10/200
15/15 [==============================] - 0s 3ms/step - loss: 0.5936 - accuracy: 0.7448
Epoch 11/

In [11]:
# 모델을 테스트 데이터 셋에 적용 >> 정확도 구하기
model.evaluate(x_test, y_test)
#  loss 와 accuracy 두 가지를 계산
# model.evaluate 함수는 Keras와 같은 딥러닝 프레임워크에서 모델의 성능을 평가할 때 사용하는 함수
# model.evaluate 함수는 모델이 학습되지 않은 데이터에서 얼마나 잘 작동하는지 평가하는 데 유용

2/2 [==============================] - 0s 7ms/step - loss: 0.6235 - accuracy: 0.8413


[0.623489499092102, 0.841269850730896]

In [12]:
score = model.evaluate(x_test, y_test)
print('Test Data loss = ', round(score[0] * 100 , 2))
print('Test Data accuracy = ', round(score[1]*100,2))

2/2 [==============================] - 0s 7ms/step - loss: 0.6235 - accuracy: 0.8413
Test Data loss =  62.35
Test Data accuracy =  84.13


In [13]:
# 모델 저장
path = ('../mulcam_bigdata/09_DL/sonar.hdf5')
model.save(path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [14]:
del model

In [15]:
# 모델 재사용
model = load_model('../mulcam_bigdata/09_DL/sonar.hdf5')

In [16]:
score = model.evaluate(x_test, y_test)
print('Test Data loss = ', round(score[0] * 100 , 2))
print('Test Data accuracy = ', round(score[1]*100,2))

2/2 [==============================] - 0s 7ms/step - loss: 0.6235 - accuracy: 0.8413
Test Data loss =  62.35
Test Data accuracy =  84.13


k겹 교차 검증

- 테이터셋을 여러 개로 나누어 하나씩 테스트셋으로 사용하고 나머지를 모두 합해서 학습셋으로 사용하는 방법
- 가지고 있는 데이터의 100%를 학습셋으로 사용할 수 있고, 또 동시에 테스트셋으로도 사용할 수 있음

In [18]:
# 교차 검증(KFold)
from sklearn.model_selection import KFold

x = df.iloc[:, :-1]   # 1~60번째 열을 x 변수에 저장
y = df.iloc[:, -1]    # 광물의 종류는 y로 저장

k = 5 # 5개의 폴드로 나눔

kfold = KFold(n_splits = k, shuffle = True)   # 샘플이 어느 한쪽에 치우치지 않도록 shuffle 옵션을 True로 설정

# 정확도를 담을 그릇(리스트) 준비
acc_score = []

# 모델구조설정함수
def model_fn():
  model = Sequential()    # 딥러닝 모델의 구조
  model.add(Dense(24, input_dim = 60, activation = 'relu'))
  model.add(Dense(10, activation = 'relu'))
  model.add(Dense(1, activation = 'sigmoid'))
  return model

# k겹 교차 검증 이용, k번 학습 실행
# for 문에 의해 k번 반복
# split()에 의해 k개의 학습셋, 테스트셋으로 분리
for train_index, test_index in kfold.split(x):
  x_train, x_test = x.iloc[train_index, :], x.iloc[test_index, :]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  # 모델 생성
  model = model_fn()

  # 모델 컴파일
  model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  history = model.fit(x_train, y_train, epochs = 200, batch_size = 10, verbose = 0)
  # verbose = 0: 출력되는 정보를 최소화

  accuracy = model.evaluate(x_test, y_test)[1]  # 정확도 구하기
  acc_score.append(accuracy)                    # 정확도 리스트에 저장

  # k번 실행된 정확도 평균 구하기
  avg_acc_score = sum(acc_score)/k

  # 결과출력
  print('정확도: ', acc_score)
  print('kfold 평균 정확도: ', avg_acc_score)

2/2 [==============================] - 0s 8ms/step - loss: 0.4060 - accuracy: 0.9286
정확도:  [0.9285714030265808]
kfold 평균 정확도:  0.18571428060531617
2/2 [==============================] - 0s 7ms/step - loss: 0.4492 - accuracy: 0.8333
정확도:  [0.9285714030265808, 0.8333333134651184]
kfold 평균 정확도:  0.35238094329833985
2/2 [==============================] - 0s 8ms/step - loss: 0.6746 - accuracy: 0.8333
정확도:  [0.9285714030265808, 0.8333333134651184, 0.8333333134651184]
kfold 평균 정확도:  0.5190476059913636
2/2 [==============================] - 0s 8ms/step - loss: 0.4911 - accuracy: 0.8780
정확도:  [0.9285714030265808, 0.8333333134651184, 0.8333333134651184, 0.8780487775802612]
kfold 평균 정확도:  0.6946573615074157
2/2 [==============================] - 0s 8ms/step - loss: 0.7521 - accuracy: 0.8049
정확도:  [0.9285714030265808, 0.8333333134651184, 0.8333333134651184, 0.8780487775802612, 0.8048780560493469]
kfold 평균 정확도:  0.8556329727172851
